# Using BuildAMol and DiffDock together

`DiffDock` is a deep learning tool to perform molecular docking from PDB files for the target proteins and some input that can be read into an RDKit Molecule for the ligands. Here we demonstrate how we could use BuildAMol in a project that tries to evaluate different ligands using DiffDock. 

> Disclaimer:
> We actually altered parts of the source code for DiffDock to get a smoother workflow, and developed our own wrapper API to use DiffDock in a programming setting instead of as a command-line tool. 

## Example
Let's say we want to see where a specific ligand would bind depending on what kind of modification we perform. For instance, let's add a specific functional group at one position and observe how this changes the predicted binding behaviour using DiffDock!

For this example we will again use the DRD2 protein, which we already used in our main ligand-design tutorial from the documentation.

In [2]:
import buildamol as bam
from pathlib import Path
import diffdock_api as diffdock # our own API wrappers

# initialize DiffDock
diffdock.init(Path("../GIT/DiffDock").resolve())

Now that we have successfully set up DiffDock for use, we can load our protein:

In [3]:
diffdock.load_protein("../GIT/biobuild/docs/examples/files/DRD2.pdb")

And once this is done, we can now start thinking about our ligand. Let's say we keep using the ligand we generated in the former tutorial, but now simply add one functional group to it to observe the difference. Here's how we will do it:

In [4]:
ligand = bam.read_pdb("../GIT/biobuild/docs/examples/files/DRD2_ligand.pdb")
ligand.add_hydrogens()
ligand.py3dmol().show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Let's say we try adding different functional groups onto the methyl group in order to observe the docking behaviour!

In [5]:
# define some modifications we want to try out
modifiers = [bam.acetylate, bam.phenolate, bam.aminate, bam.phosphorylate, bam.carboxylate]

# identify the target methyl-carbon
target = ligand.search_by_constraints([bam.structural.neighbors.constraints.has_neighbor_hist({"H": 3})])
target = target[0][0]

# now cast the modifications over the ligand
modified = [modifier(ligand, at_atom=target, inplace=False) for modifier in modifiers]

In [6]:
colors = ["magenta", "green", "cyan", "orange", "purple"]
viewer = ligand.py3dmol()
for c, m in zip(colors, modified):
    viewer.add(m.py3dmol(color=c))
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Alrighty, so with this done, let's dock each of our modified versions (as well as the original, of course) a few times. We can do so by using:

In [8]:
ligands = [ligand] + modified

# the number of conformers we want to generate when docking
n_conformers = 3

generated = []
for lig in ligands:
    # load the ligand into diffdock 
    # (this needs to be via an RDKit molecule
    # which must not have any hydrogens)
    lig = lig.copy()
    lig -= lig.get_atoms("H", by="element")
    lig = lig.to_rdkit()
    diffdock.load_ligand(lig)

    # now perform docking
    conformers, confidence_scores = diffdock.dock(n_conformers)
    generated.append((conformers, confidence_scores))

@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 3124 atoms and 1 coordinate set(s) were parsed in 0.03s.


And now let's actually get a look at what we got! The `conformers` are a list of RDKit molecules which we can convert back to BuildAMol for easier handling:

In [9]:
_generated = [[bam.Molecule.from_rdkit(i) for i in j] for j, _ in generated]

In [10]:
import py3Dmol as p3d

view = p3d.view(width=800, height=600)
view.addModels(open("../GIT/biobuild/docs/examples/files/DRD2.pdb").read(), "pdb")
view.setStyle({"model":0}, {"cartoon": {"color": "spectrum"} })
view.adddSurface(None, {"opacity": 1, "color": "lightgray"})

idx = 1
for color, _list in zip(colors, _generated):
    for mol in _list:
        view.addModels(bam.utils.pdb.encode_pdb(mol), "pdb")
        view.setStyle({"model": idx}, {"stick" : {"color": color}})
        idx += 1

view.ZoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

And finally, let's save the generated molecules to PDB files and write a text file with their confidence scores:

In [37]:
data = []
out_dir = Path("./out").resolve()

if not out_dir.exists():
    out_dir.mkdir()
labels = ["none"] + [i.__name__ for i in modifiers]

for i in range(len(generated)):
    _mols = _generated[i]
    modifier = labels[i]
    confidences = generated[i][1][:, 0].cpu().numpy()
    for idx, mol in enumerate(_mols):
        conf = confidences[idx]
        fname = f"{modifier}_{idx}.pdb"
        data.append(f"{fname}, {conf}")
        mol.to_pdb(out_dir / fname)

with open(out_dir / "confidences.csv", "w") as f:
    f.write("\n".join(data))
    